In [ ]:
#Importing Libraries
import pandas as pd

#Nan-DecreaseCount
mono=pd.read_excel("./volume_trend_month_sector_SOLDTO.xlsx","1884_soldto")
orig=mono.copy()
months_s = ['Volume_7', 'Volume_8','Volume_9','Volume_10','Volume_11']
mono["Nan_values_count"]=mono.isnull().sum(axis=1)
mono=mono.fillna(0)
mono["decrease"]=mono[months_s].apply(lambda x: x.is_monotonic_decreasing, axis=1)

#Percentage Change
perc_change=mono[months_s].pct_change(axis="columns")
perc_change["SOLDTO"]=mono["SOLDTO"]
perc_change["negative_values"]=perc_change.T[perc_change.T<0].count()
infinity_check=mono[months_s][np.isinf(perc_change)].combine_first(df)
mono["negative_values"]=(infinity_check["SOLDTO"]==mono["SOLDTO"])*infinity_check["negative_values"]

In [1]:
import nbdev


# Modelling-1st Iteration

In [2]:
#export
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import eli5


In [3]:
inactive=pd.read_excel("volume_trend_month_1to6.xlsx","Inactive_perc_8-6",header=None)
active=pd.read_excel("volume_trend_month_1to6.xlsx","active_perc_model_1-11",header=None)


inact_act = pd.concat([inactive,active], ignore_index=True)
inact_act=inact_act.iloc[1:, :]
inact_act=inact_act[inact_act[11]!='Category']
inact_act=inact_act.fillna(0)


In [4]:
# from sklearn import preprocessing
le=LabelEncoder()

inact_act[11]=le.fit_transform(inact_act[11])
X = inact_act[[1,2,3,4,5,6,7,8,9,10]]
y = inact_act[11]

In [5]:
le.classes_

array(['Active', 'Inactive'], dtype=object)

In [6]:
# split the dataset into the training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
# build the lightgbm model

clf = lgb.LGBMClassifier()
clf.fit(X_train.astype(float), y_train)
# view accuracy


LGBMClassifier()

In [7]:
y_pred=clf.predict(X_test.astype(float))
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.8783


In [8]:
# view confusion-matrix
# Print the Confusion Matrix and slice it into four pieces


cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[441  18]
 [ 47  28]]

True Positives(TP) =  441

True Negatives(TN) =  28

False Positives(FP) =  18

False Negatives(FN) =  47


In [9]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       459
           1       0.61      0.37      0.46        75

    accuracy                           0.88       534
   macro avg       0.76      0.67      0.70       534
weighted avg       0.86      0.88      0.87       534



In [10]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))
  
# import SMOTE module from imblearn library
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

Before OverSampling, counts of label '1': 205
Before OverSampling, counts of label '0': 1395 

After OverSampling, the shape of train_X: (2790, 10)
After OverSampling, the shape of train_y: (2790,) 

After OverSampling, counts of label '1': 1395
After OverSampling, counts of label '0': 1395


In [11]:
clf = lgb.LGBMClassifier()
clf.fit(X_train_res, y_train_res.ravel())
predictions = clf.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       459
           1       0.52      0.57      0.54        75

    accuracy                           0.87       534
   macro avg       0.72      0.74      0.73       534
weighted avg       0.87      0.87      0.87       534



In [12]:
eli5.show_weights(clf)


Weight,Feature
0.3475,10
0.1052,9
0.0883,7
0.0790,8
0.0749,5
0.0745,6
0.0732,1
0.0637,2
0.0551,3
0.0387,4


# 2ND ITERATION

In [13]:
inactive=pd.read_excel("volume_trend_month_1to6.xlsx","Inactive_perc_model_1-6",header=None)
active=pd.read_excel("volume_trend_month_1to6.xlsx","active_perc_model_1-6",header=None)


inact_act = pd.concat([inactive,active], ignore_index=True)
inact_act=inact_act.iloc[1:, :]
inact_act=inact_act[inact_act[6]!='Category']
inact_act=inact_act.fillna(0)


# from sklearn import preprocessing
le=LabelEncoder()

inact_act[6]=le.fit_transform(inact_act[6])
X = inact_act[[1,2,3,4,5]]
y = inact_act[6]

le.classes_



array(['Active', 'Inactive'], dtype=object)

In [14]:
# split the dataset into the training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
# build the lightgbm model

clf = lgb.LGBMClassifier()
clf.fit(X_train.astype(float), y_train)
# view accuracy


y_pred=clf.predict(X_test.astype(float))
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

# view confusion-matrix
# Print the Confusion Matrix and slice it into four pieces


cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

print(classification_report(y_test, y_pred))



LightGBM Model accuracy score: 0.8970
Confusion matrix

 [[440  19]
 [ 36  39]]

True Positives(TP) =  440

True Negatives(TN) =  39

False Positives(FP) =  19

False Negatives(FN) =  36
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       459
           1       0.67      0.52      0.59        75

    accuracy                           0.90       534
   macro avg       0.80      0.74      0.76       534
weighted avg       0.89      0.90      0.89       534



In [15]:
eli5.show_weights(clf)


Weight,Feature
0.3174,5
0.2173,1
0.2028,2
0.1526,3
0.1099,4


In [16]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))
  
# import SMOTE module from imblearn library
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

clf = lgb.LGBMClassifier()
clf.fit(X_train_res, y_train_res.ravel())
predictions = clf.predict(X_test)
  
# print classification report
print(classification_report(y_test, predictions))



Before OverSampling, counts of label '1': 205
Before OverSampling, counts of label '0': 1395 

After OverSampling, the shape of train_X: (2790, 5)
After OverSampling, the shape of train_y: (2790,) 

After OverSampling, counts of label '1': 1395
After OverSampling, counts of label '0': 1395
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       459
           1       0.55      0.69      0.62        75

    accuracy                           0.88       534
   macro avg       0.75      0.80      0.77       534
weighted avg       0.89      0.88      0.88       534



In [17]:
eli5.show_weights(clf)


Weight,Feature
0.3262,5
0.1951,1
0.1857,2
0.1510,4
0.1420,3


# Train-Test-Validation

In [18]:
# set aside 20% of train and test data for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.05, random_state= 8)

In [19]:
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(y_train.shape))
print("y val shape: {}".format(y_test.shape))

X_train shape: (1621, 5)
X_test shape: (427, 5)
y_train shape: (1621,)
y_test shape: (427,)
X_val shape: (1621,)
y val shape: (427,)


In [20]:
clf = lgb.LGBMClassifier()
clf.fit(X_train.astype(float), y_train)


# view Validation  accuracy


y_pred=clf.predict(X_val.astype(float))
accuracy=accuracy_score(y_pred, y_val)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_val, y_pred)))

# view confusion-matrix
# Print the Confusion Matrix and slice it into four pieces


cm = confusion_matrix(y_val, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

print(classification_report(y_val, y_pred))


LightGBM Model accuracy score: 0.9186
Confusion matrix

 [[72  4]
 [ 3  7]]

True Positives(TP) =  72

True Negatives(TN) =  7

False Positives(FP) =  4

False Negatives(FN) =  3
              precision    recall  f1-score   support

           0       0.96      0.95      0.95        76
           1       0.64      0.70      0.67        10

    accuracy                           0.92        86
   macro avg       0.80      0.82      0.81        86
weighted avg       0.92      0.92      0.92        86



In [21]:
#Viewing Testing Accuracy
y_pred=clf.predict(X_test.astype(float))
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

# view confusion-matrix
# Print the Confusion Matrix and slice it into four pieces


cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

print(classification_report(y_test
                            , y_pred))


LightGBM Model accuracy score: 0.8946
Confusion matrix

 [[349  19]
 [ 26  33]]

True Positives(TP) =  349

True Negatives(TN) =  33

False Positives(FP) =  19

False Negatives(FN) =  26
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       368
           1       0.63      0.56      0.59        59

    accuracy                           0.89       427
   macro avg       0.78      0.75      0.77       427
weighted avg       0.89      0.89      0.89       427



In [23]:
!nbdev_prepare

Success.


'quarto' is not recognized as an internal or external command,
operable program or batch file.


# Iteration3

In [2]:
!nbdev_help


nbdev_bump_version              Increment version in settings.ini by one
nbdev_changelog                 Create a CHANGELOG.md file from closed and labeled GitHub issues
nbdev_clean                     Clean all notebooks in `fname` to avoid merge conflicts
nbdev_conda                     Create a `meta.yaml` file ready to be built into a package, and optionally build and upload it
nbdev_create_config             Create a config file.
nbdev_docs                      Create Quarto docs and README.md
nbdev_export                    Export notebooks in `path` to Python modules
nbdev_filter                    A notebook filter for Quarto
nbdev_fix                       Create working notebook from conflicted notebook `nbname`
nbdev_help                      Show help for all console scripts
nbdev_install                   Install Quarto and the current library
nbdev_install_hooks             Install Jupyter and git hooks to automatically clean, trust, and fix merge conflicts in notebooks
n